# Futher processing of "Ticket inspector logs" 

The "inspector" column is irrelevant.

In order to collapse the inspector column, I need to be able to identify individual incoming buses 



In [11]:
import pandas as pd
from datetime import datetime, timedelta
from collections import defaultdict

### Identifying individual incoming buses 

In [12]:
Inspections = pd.read_csv("C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe1/Inspections.csv", parse_dates = ["Start_Dtime", "End_Dtime"])
Inspections.head(5)

C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_3344\3635811648.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  Inspections = pd.read_csv("C:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe1/Inspections.csv", parse_dates = ["Start_Dtime", "End_Dtime"])


,Stop,Stop_code,Line,Inspector,Cards_checked,Validated,Unvalidated,Start_Dtime,End_Dtime
0,Vene,7820301-1,7,Karl Müürsepp,5,5,0,2016-02-12 06:50:33,2016-02-12 06:50:52
1,Vene,7820301-1,7,Külli Henn,3,1,2,2016-02-12 06:50:36,2016-02-12 06:51:41
2,Vene,7820301-1,7,Mariin Piper,5,4,1,2016-02-12 06:50:38,2016-02-12 06:50:59
3,Vene,7820301-1,4,Karl Müürsepp,9,6,3,2016-02-12 06:53:30,2016-02-12 07:06:12
4,Vene,7820301-1,4,Külli Henn,8,7,1,2016-02-12 06:53:33,2016-02-12 07:06:40


In [13]:
Inspections["Date"] = Inspections["Start_Dtime"].dt.date

def Bus_visit(Group):
    # List for storing where the bus changes - all of the same number means same bus 
    List = []
    Bus_id = 0
     # convert temporailiy into datetime objects - this is only done to allow for substraction
    Previous_time = Group.iloc[0]

    for New_time in Group:
        # if difference ois greater than 3 minutes - assume its a new bus 
        if (New_time - Previous_time) > timedelta(minutes = 3):
            Bus_id += 1 
        
        List.append(Bus_id)
        # and then I need to update the previous date time to the new bus time 
        Previous_time = New_time

    return pd.Series(List, index = Group.index)


Inspections["Repeat_bus"] = Inspections.groupby(["Date", "Stop_code", "Line"], group_keys = False)["Start_Dtime"].apply(Bus_visit)

# Inspecting first 40 rows - appears to be working correctly
Inspections.head(40)

,Stop,Stop_code,Line,Inspector,Cards_checked,Validated,Unvalidated,Start_Dtime,End_Dtime,Date,Repeat_bus
0,Vene,7820301-1,7,Karl Müürsepp,5,5,0,2016-02-12 06:50:33,2016-02-12 06:50:52,2016-02-12,0.0
1,Vene,7820301-1,7,Külli Henn,3,1,2,2016-02-12 06:50:36,2016-02-12 06:51:41,2016-02-12,0.0
2,Vene,7820301-1,7,Mariin Piper,5,4,1,2016-02-12 06:50:38,2016-02-12 06:50:59,2016-02-12,0.0
3,Vene,7820301-1,4,Karl Müürsepp,9,6,3,2016-02-12 06:53:30,2016-02-12 07:06:12,2016-02-12,0.0
4,Vene,7820301-1,4,Külli Henn,8,7,1,2016-02-12 06:53:33,2016-02-12 07:06:40,2016-02-12,0.0
5,Vene,7820301-1,4,Mariin Piper,12,9,3,2016-02-12 06:53:36,2016-02-12 07:06:13,2016-02-12,0.0
6,Vene,7820301-1,27,Karl Müürsepp,6,4,2,2016-02-12 07:14:40,2016-02-12 07:15:15,2016-02-12,0.0
7,Vene,7820301-1,27,Külli Henn,5,5,0,2016-02-12 07:14:48,2016-02-12 07:15:01,2016-02-12,0.0
8,Vene,7820301-1,27,Mariin Piper,4,4,0,2016-02-12 07:14:49,2016-02-12 07:15:04,2016-02-12,0.0
9,Vene,7820301-1,4,Karl Müürsepp,11,9,2,2016-02-12 07:24:44,2016-02-12 07:25:39,2016-02-12,1.0


In [7]:
# Collapsing the unnecasary dimension of the ticket inspector

Collapsed = Inspections.groupby(["Date", "Stop", "Stop_code", "Line", "Repeat_bus"]).agg({"Cards_checked" : "sum",
                                                               "Validated" : "sum",
                                                               "Unvalidated" : "sum",
                                                               "Start_Dtime" : "first",
                                                               "End_Dtime" : "last",
                                                              }).reset_index()

Collapsed.head()

,Date,Stop,Stop_code,Line,Repeat_bus,Cards_checked,Validated,Unvalidated,Start_Dtime,End_Dtime
0,2016-02-12,Kalda tee,7820064-1,1,0.0,12,12,0,2016-02-12 13:16:51,2016-02-12 13:17:17
1,2016-02-12,Kalda tee,7820064-1,1,1.0,19,18,1,2016-02-12 13:33:31,2016-02-12 13:34:12
2,2016-02-12,Kalda tee,7820064-1,15,0.0,6,5,1,2016-02-12 13:50:55,2016-02-12 13:51:11
3,2016-02-12,Kalda tee,7820064-1,20,0.0,37,23,14,2016-02-12 13:12:26,2016-02-12 13:14:13
4,2016-02-12,Kalda tee,7820064-1,26,0.0,20,11,9,2016-02-12 13:19:15,2016-02-12 13:20:15


### Distinguishing between the same bus stop being visit multiple times in a single day 

In [14]:
# I need to be able to distinguish between the same bus stop being visit multiple times in a single day 
# Condition 1 : That there is an bus stop check for an intervening bus stop
# Condition 2 : That the difference between the previous and current entry (for the same bus stop) is greater than 30 minutes 
# I chose greater than 30 minutes because often the ticket inspectors split them selves up across two close bus stops


def Repeat_visit(subset):

    # Track: last end-time seen per stop_code
    last_end_time = {}

    # Track: visit counter per stop_code
    visit_count = {}

    # Output list
    visit_index_list = []

    # Track last *row's* stop code for Condition 1
    prev_stop_code = None

    for idx, row in subset.iterrows():
        stop = row["Stop_code"]
        start_t = row["Start_Dtime"]
        end_t = row["End_Dtime"]

        # Retrieve last known time this stop was seen
        prev_end_for_this_stop = last_end_time.get(stop)

        # FIRST TIME EVER SEEING THIS STOP
        if prev_end_for_this_stop is None:
            visit_count[stop] = 1
            visit_index_list.append(1)

        else:
            # CONDITION 1 + CONDITION 2
            cond1 = (stop != prev_stop_code)
            cond2 = (start_t - prev_end_for_this_stop) > timedelta(minutes=30)

            if cond1 and cond2:
                # New visit
                visit_count[stop] = visit_count.get(stop, 1) + 1
                visit_index_list.append(visit_count[stop])
            else:
                # Same visit as before
                visit_index_list.append(visit_count[stop])

        # Update last end time for this stop
        last_end_time[stop] = end_t

        # Update previous stop code for next iteration
        prev_stop_code = stop


    return pd.Series(visit_index_list, index = subset.index)


Collapsed["Visit_number"] = Collapsed.groupby(["Date"]).apply(Repeat_visit).droplevel(0)

C:\Users\Henry Chapman\AppData\Local\Temp\ipykernel_3344\3317126212.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  Collapsed["Visit_number"] = Collapsed.groupby(["Date"]).apply(Repeat_visit).droplevel(0)


In [15]:
# This means that over the last 10 years, ~60,000 buses have been inspected. 
len(Collapsed)

60798

### Adding in pre / post 2019 column 

As eluded to, in 2019, the bus network underwent a major upheaval 

https://tartu.ee/et/uudised/1-juulist-muudavad-mitmed-bussipeatused-nime-voi-asukohta

Inlcuding the addition of 27 new stops, removal of 10 stops, renaming of 33 stops 

Reduction of the number of buses from from 29 to 20

This makes the same bus stop, pre and post 2019 upheaval no longer comparable

I can still use them in training, but I will need to ensure I differential between pre and post 2019 

The exact date of the switch was : 2019-07-01

In [18]:
Collapsed["Date"] = pd.to_datetime(Collapsed["Date"], format = "%Y-%m-%d")


real_switch = pd.Timestamp("2019-07-01")

Collapsed["Pre_2019"] = Collapsed["Date"] < real_switch

Collapsed.head()

,Date,Stop,Stop_code,Line,Repeat_bus,Cards_checked,Validated,Unvalidated,Start_Dtime,End_Dtime,Visit_number,Pre_2019
0,2016-02-12,Kalda tee,7820064-1,1,0.0,12,12,0,2016-02-12 13:16:51,2016-02-12 13:17:17,1,True
1,2016-02-12,Kalda tee,7820064-1,1,1.0,19,18,1,2016-02-12 13:33:31,2016-02-12 13:34:12,1,True
2,2016-02-12,Kalda tee,7820064-1,15,0.0,6,5,1,2016-02-12 13:50:55,2016-02-12 13:51:11,1,True
3,2016-02-12,Kalda tee,7820064-1,20,0.0,37,23,14,2016-02-12 13:12:26,2016-02-12 13:14:13,1,True
4,2016-02-12,Kalda tee,7820064-1,26,0.0,20,11,9,2016-02-12 13:19:15,2016-02-12 13:20:15,1,True


In [25]:
real_switch = pd.Timestamp("2019-07-01")

Collapsed["Pre_2019"] = Collapsed["Start_Dtime"] < real_switch
Collapsed.head()

,Date,Stop,Stop_code,Line,Repeat_bus,Cards_checked,Validated,Unvalidated,Start_Dtime,End_Dtime,Visit_number,Pre_2019
0,2016-02-12,Kalda tee,7820064-1,1,0.0,12,12,0,2016-02-12 13:16:51,2016-02-12 13:17:17,1,True
1,2016-02-12,Kalda tee,7820064-1,1,1.0,19,18,1,2016-02-12 13:33:31,2016-02-12 13:34:12,1,True
2,2016-02-12,Kalda tee,7820064-1,15,0.0,6,5,1,2016-02-12 13:50:55,2016-02-12 13:51:11,1,True
3,2016-02-12,Kalda tee,7820064-1,20,0.0,37,23,14,2016-02-12 13:12:26,2016-02-12 13:14:13,1,True
4,2016-02-12,Kalda tee,7820064-1,26,0.0,20,11,9,2016-02-12 13:19:15,2016-02-12 13:20:15,1,True


# Exporting 

In [27]:
# Neccasary colunms 

keep_cols = ["Date", "Start_Dtime", "End_Dtime", "Stop_code", "Line", "Repeat_bus", "Visit_number", "Unvalidated", "Pre_2019"]

Final = Collapsed[keep_cols]

Final.to_csv("c:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe2/Inspections.csv", index = False)

In [28]:
Final.head()

,Date,Start_Dtime,End_Dtime,Stop_code,Line,Repeat_bus,Visit_number,Unvalidated,Pre_2019
0,2016-02-12,2016-02-12 13:16:51,2016-02-12 13:17:17,7820064-1,1,0.0,1,0,True
1,2016-02-12,2016-02-12 13:33:31,2016-02-12 13:34:12,7820064-1,1,1.0,1,1,True
2,2016-02-12,2016-02-12 13:50:55,2016-02-12 13:51:11,7820064-1,15,0.0,1,1,True
3,2016-02-12,2016-02-12 13:12:26,2016-02-12 13:14:13,7820064-1,20,0.0,1,14,True
4,2016-02-12,2016-02-12 13:19:15,2016-02-12 13:20:15,7820064-1,26,0.0,1,9,True


### Exporting all stop_IDs that have been visited

In [29]:
Inspections_stop_codes = pd.Series(Final["Stop_code"].unique())
Inspections_stop_codes.to_csv("c:/users/henry chapman/Documents/Coding/Data_science/Project_final/Output/1_Compiling_data/Pipe2/Inspections_stop_codes.csv", index = False)

In [30]:
Inspections_stop_codes.head()

0    7820064-1
1    7820068-1
2    7820126-1
3    7820140-1
4    7820153-1
dtype: object